In [40]:
from Brands import Sonites
from Analyzer import Analyzer
from Solver import Solver
from Utils import combined_error_minimum_distance
import numpy as np

son = Sonites()
an = Analyzer()
solver = Solver()



./Exports/TeamExport_A46051_Alpha_M_Period 3.xlsx loaded
No weights provided
before
./Exports/TeamExport_A46051_Alpha_M_Period 3.xlsx loaded
after
./Exports/TeamExport_A46051_Alpha_M_Period 3.xlsx loaded
Attributes file:./Attributes/attributes_3.json


In [33]:
df_seg_sem_fc = an.forecast_df(dataframe=son.df_segments_semantic, steps=2)
target_sem = df_seg_sem_fc[(df_seg_sem_fc["Segment"]=="Explorers")&(df_seg_sem_fc["Period"]==5)][['# Features', 'Design Index', 'Battery Life', 'Display Size', 'Proc. Power', 'Price']].values
target_sem = target_sem.flatten()
target_sem

array([4.489999999999991, 2.7399999999999896, 5.42999999999998,
       5.489999999999952, 5.909999999999947, 4.39999999999999],
      dtype=object)

In [34]:
df_seg_mds_fc = an.forecast_df(dataframe=son.df_segments_mds, steps=2)
target_mds = df_seg_mds_fc[(df_seg_mds_fc["Segment"]=="Explorers")&(df_seg_mds_fc["Period"]==5)][['Economy', 'Performance', 'Convenience']].values
target_mds = target_mds.flatten()
target_mds

array([-2.9, 12.199999999999974, -3.8200000000000056], dtype=object)

In [41]:
features = son.move_features.drop(labels="Base Cost", axis=1)
features = features.values.flatten()
features

array([ 13,   6,  82,  31,  83, 350], dtype=int64)

In [43]:
solver.regress_semantic(features)

array([4.17487564, 3.30112158, 6.06721311, 5.41784812, 5.49968237,
       4.2407576 ])

In [37]:
son.move_semantic

,# Features,Design Index,Battery Life,Display Size,Proc. Power,Price
MARKET : Sonites,,,,,,
MOVE,4.26,2.58,5.93,4.56,4.58,4.71


In [ ]:
import optuna

def find_optimum_constrained(ideal_semantic: list, ideal_mds: list, semantic_weights: list,
                            mds_weights: list = [1/3, 1/3, 1/3], error_weights: np.ndarray = np.array([1, 1])):
    # Define Feature bounds
    feature_bounds = [(5, 20), (3, 10), (24, 96), (4, 40), (5, 100), (215, 475)]

    def objective(trial):
        # Sample features within the defined bounds
        features = [trial.suggest_float(f'feature_{i}', bound[0], bound[1]) for i, bound in enumerate(feature_bounds)]
        
        # Calculate the combined error using the original function (assumed to be defined outside this context)
        combined_error = combined_error(
            features,
            ideal_semantic, 
            ideal_mds, 
            semantic_weights, 
            mds_weights, 
            error_weights, 
            solver
        )
        return combined_error

    # Create an Optuna study and optimize the objective function
    study = optuna.create_study()
    study.optimize(objective, n_trials=1000)  # You can adjust the number of trials as needed

    optimal_features = study.best_params.values()
    min_error = study.best_value
    print("Optimal Features:", optimal_features)
    print("Minimum Combined Error:", min_error)

    return optimal_features, min_error
